In [ ]:

%matplotlib inline
import pyarrow as pa
import pandas as pd
import pyarrow.compute as pc
from pyarrow import csv
import numpy as np

from data_pipeline_homebrew.pandas import pandas_pipeline as pp
from importlib import reload
import io
import requests

reload(pp)



url:str = r"https://oedi-data-lake.s3.amazonaws.com/nrel-pds-building-stock/end-use-load-profiles-for-us-building-stock/2024/resstock_tmy3_release_2/timeseries_aggregates/by_ashrae_iecc_climate_zone_2004/upgrade=0/ashrae_iecc_climate_zone_2004=1A/up00-1a-multi-family_with_2_-_4_units.csv"


csv_data = pp.LocalCSVDataSource(url)

def fuel_summation_schema_function(schema: pp.PandasSchema):
    return pp.PandasSchema(
        {
            'electricity': np.float64,
            'natural_gas': np.float64,
            'fuel_oil': np.float64,
        }
    )

def fuel_summation_function(data: pd.DataFrame):
    fuels = [
        'electricity',
        'natural_gas',
        'fuel_oil',
    ]
    fuel_columns = {
        fuel: [col for col in data.columns if fuel in col]
        for fuel in fuels
    }

    data_dict = {fuel: data[fuel_columns[fuel]].sum(axis=1)
        for fuel, cols in fuel_columns.items()
    }
    return pd.DataFrame(data_dict)

fuel_summation_transformer = pp.PandasTransformer(
    transform_function=fuel_summation_function,
    schema_function=fuel_summation_schema_function
)

builder = pp.PandasMatplotlibPipelineBuilder.empty()

builder.add_source(source=csv_data, stack_id="0")
builder.add_transformer(transformer=fuel_summation_transformer, target_stack_id="1", source_stack_id="0")
builder.add_artifact(stack_id="1", artifact_class=pp.SimpleLinePlot)
builder.add_artifact(stack_id="0", artifact_class=pp.SimpleLinePlot)

builder.generate(), builder.stacks






In [ ]:
import pandas as pd
df = pd.read_csv(ResstockDataSource().url)
df

In [ ]:
fuels = FuelSummationTransformer().end_uses
fuel_sums = {
    fuel: df[[col for col in df.columns if fuel in col]].sum(axis=1)
    for fuel in fuels
}
pd.DataFrame(fuel_sums).plot()

In [ ]:
!pip install requests